In [ ]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import load_model

import numpy as np
import matplotlib.pyplot as plt
from time import time
from random import randint

from keras.datasets import cifar10

from keras.utils import np_utils
from keras.constraints import maxnorm
from keras.layers import Dense, Input
from keras.layers import Dropout
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.models import Model
from keras.callbacks import TensorBoard

from keras import regularizers
from keras import backend as K



Using TensorFlow backend.


In [ ]:

# downloading data from internet
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [ ]:
# normalize inputs from 0-255 to 0.0-1.0
x_train = x_train.astype('uint8') / 255.
x_test = x_test.astype('uint8') / 255.

In [ ]:
#Reshaping the data into 2d
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

(50000, 3072)
(10000, 3072)


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
standardscaler = StandardScaler()
pca = PCA()
pca.fit(x_train)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [ ]:
x_train_pca = pca.transform(x_train)
x_test_pca = pca.transform(x_test)
x_train_pca = x_train_pca.reshape(-1, 32,32,3)
x_test_pca = x_test_pca.reshape(-1, 32,32,3)

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32,32,3), kernel_constraint=maxnorm(3)))
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))


In [ ]:
epochs = 20
lrate = 0.01
decay = lrate/epochs
sgd = keras.optimizers.SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=sgd, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               4194816   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0

In [9]:
history2 = model.fit(x_train_pca, y_train, validation_data=(x_test_pca, y_test), epochs=epochs, batch_size=32)


Epoch 1/20
1563/1563 [==============================] - 8s 5ms/step - loss: 1.9760 - accuracy: 0.4862 - val_loss: 1.9758 - val_accuracy: 0.4831
Epoch 2/20
1563/1563 [==============================] - 8s 5ms/step - loss: 1.9727 - accuracy: 0.4901 - val_loss: 1.9767 - val_accuracy: 0.4842
Epoch 3/20
1563/1563 [==============================] - 8s 5ms/step - loss: 1.9732 - accuracy: 0.4888 - val_loss: 1.9742 - val_accuracy: 0.4845
Epoch 4/20
1563/1563 [==============================] - 8s 5ms/step - loss: 1.9706 - accuracy: 0.4909 - val_loss: 1.9747 - val_accuracy: 0.4848
Epoch 5/20
1563/1563 [==============================] - 8s 5ms/step - loss: 1.9686 - accuracy: 0.4934 - val_loss: 1.9731 - val_accuracy: 0.4874
Epoch 6/20
1563/1563 [==============================] - 8s 5ms/step - loss: 1.9677 - accuracy: 0.4941 - val_loss: 1.9730 - val_accuracy: 0.4875
Epoch 7/20
1563/1563 [==============================] - 8s 5ms/step - loss: 1.9656 - accuracy: 0.4972 - val_loss: 1.9699 - val_accuracy:

In [11]:
scores = model.evaluate(x_test_pca, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 49.50%
